In [1]:
import math
import numpy as np

### Montgomeries Transformation

In [2]:
def MTransformation(a,p):
    exp = math.ceil(math.log(p, 2))
    R = pow(2, exp)
    return (a*R)%p

In [3]:
print(MTransformation(2,11))

10


## MMM

In [14]:
def inv(a, p):
    for i in range(1, p):
        if (a*i)%p == 1:
            return i

In [5]:
def MMM(a,b,p):
    return MTransformation(a*b, p)

In [6]:
print(MMM(2,5,7))

3


## MMM reduction

In [23]:
def find_v(p):
    exp = math.ceil(math.log(p, 2))
    R = pow(2, exp)
    Rinv = inv(R, p)
    v = (R*Rinv-1)/p
    return (R, int(v))

In [28]:
def MMMr(a, b, p):
    D = MTransformation(a, p)*MTransformation(b, p)
    R, v = find_v(p)
    m = ((D%R)*v)%R
    U = (D+m*p)/R
    C = U
    if U >= p:
        C = U-p
    return int(C)

In [29]:
print(MMMr(6,14, 23))

20
